In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
#import fabio
import sys
import h5py

sys.path.insert(0, '../scripts/')
#from integrators import*
#from utilities import *
from files_tools import*


Set directory

In [8]:
# save teh path of the working directory
%cd ..
path = !pwd
path = path[0]
path

/Users/maddalena/Documents/01-SU/beamtimes/21-04-DESY-P10/02-tests


'/Users/maddalena/Documents/01-SU/beamtimes/21-04-DESY-P10/02-tests'

In [9]:
 # path to old data for comparison
path2 = '/Users/maddalena/documents/02-UniPD/02-LM Physics/2020-stockholm/master-project/01-data-analysis/XRD/new-analysis/source'

# Load data

In [16]:
!ls 01-xrd/source

calibrator-2021-03-16.hdf5 mask-d2.npy
calibrator-2021-03-19.hdf5 mask.npy
centers-d2.npy             q-d2.npy
centers.npy                q.npy


In [10]:
temp = [295, 290, 280, 270, 260, 250, 240, 230, 220, 210, 200, 190, 180]


In [17]:
q = np.load('01-xrd/source/q.npy')
q_old = np.loadtxt(path2+'/q_0114.txt')

In [36]:
# new data 2020-12-07
f = h5py.File(path+'/01-xrd/source/calibrator-2021-03-16.hdf5', 'r')
f.visititems(visit_func)   

#bg = np.asarray(f['/tool/background'])
#Lys_w = np.asarray(f['/Iq/Lys200_wei'])
#Lys_v = np.asarray(f['/Iq/Lys200_vol'])
#Solv = np.asarray(f['/Iq/Wat-Gly_11v'])
f.close()


/2d-pattern
	 /2d-pattern/bg (1024, 1024)
	 /2d-pattern/mask (1024, 1024)
	 /2d-pattern/no_bg (1024, 1024)
	 /2d-pattern/with_bg (1024, 1024)
/Iq
	 /Iq/Iq (512,)
	 /Iq/Iq2 (500,)
	 /Iq/q0114 (512,)
	 /Iq/q2 (500,)
/info
	 /info/detector-sample ()
	 /info/energy ()
	 /info/peak-radius ()
	 /info/pixel_size ()
	 /info/xy-center (2,)


In [ ]:
ls /Users/maddalena/OneDrive\ -\ Stockholm\ University/02-UniPD/02-LM\ Physics/2020-stockholm/master-project/01-data-analysis/XRD/new-analysis/source

In [ ]:
# water
f1 = h5py.File(path2+'/ang_avg_2019-11-29.hdf5', 'r')
f1.visititems(visit_func)
#Iq_DryPow_n = f1['/Dry/wtc'][:]
#Iq_HydPow_n = f1['/Hyd/wtc'][:]
Iq_wat = np.asarray(f1['/water/Iq_with_bg'])
f1.close()

In [ ]:
# 2020-01-14
f1 = h5py.File(path2+'/ang_avg_norm_data.hdf5', 'r')
#f1.visititems(visit_func)
Iq_DryPow_n = f1['/Dry/wtc'][:]
Iq_HydPow_n = f1['/Hyd/wtc'][:]

f1.close()

In [ ]:
# import old data: 2020-01-14
f = h5py.File(path2+'/ang_avg_raw_data.hdf5', 'r')
#f.visititems(visit_func)   
Iq_DryPow = np.asarray(f['/Dry/diff_T/ctw/no_bg'])
Iq_HydPow = np.asarray(f['/Hyd/diff_T/wtc/no_bg'])
Iq_HydPow_bg = np.asarray(f['/Hyd/diff_T/wtc/with_bg'])

f.close()

In [ ]:
# 2019-12-12 freezing sample

f = h5py.File(path2+'/ang_avg_2019-12-12_freezing.hdf5', 'r')
#f.visititems(visit_func) 
Iq_FHydPow_n = f['/Iq/Iq_no_bg_norm'][:]
im2d_f_bg = f['/im2d/im2d_bg'][:]
im2d_f = f['/im2d/im2d_nobg'][:]
f.close()


# Background subtraction

In [ ]:
def subtr_offset(Iq, Iq_bg, w=1.):
    off = Iq_bg[0] - Iq[0]
    Iq_npbg = Iq - w*(Iq_bg-off)
    
    return Iq_npbg

In [ ]:
!ls data/2021-01-14

In [ ]:
temp

In [ ]:
T = np.array([295,240,220,200,180,180,180,180])
T2 = np.array([295,240,220,200,180])
T3 = np.array([295,240,240,220])

# Plots

## Water/Glycerol 1:1 volume

In [ ]:
plt.figure(figsize=(9,4))

plt.subplot(121)
col = plt.cm.jet(np.linspace(1,0,len(Lys_v)+1))
for i in range(0,len(Lys_v)):
    if i in (0,1,2,3,4,7):#!= 5 or i != 6:
        #Iq = ang_and_plot(path+'/data/2021-01-14/%s' %WatGlyVol[i])
        plt.plot(q, Lys_v[i], c=col[i], lw=1, label='%d K' %T[i])

#for i in range(len(Ih_ref)-4):
#    plt.axvline(x=Ih_ref[i], c='gray', lw=1, alpha=1,ls='-.')
plt.xlabel(r'Q [$\rm\AA^{-1}$]', fontsize=14)
plt.ylabel(r'I(Q)', fontsize=14)
plt.xlim(q[0], q[-1])
#plt.ylim(0, 1.2)
plt.legend(frameon=False)
#plt.title('lys 200mg/ml in Wat/Gly 1:1 vol')

plt.subplot(122)
col = plt.cm.jet(np.linspace(1,0,len(Solv)+1))
for i in range(0,len(Solv)):
    #Iq = ang_and_plot(path+'/data/2021-01-14/%s' %Lys200Vol[i])
    plt.plot(q, Solv[i], c=col[i], lw=1, label=f'{T2[i]} K')
plt.plot(q,Iq_wat, 'k--', label='water')
plt.xlabel(r'Q [$\rm\AA^{-1}$]', fontsize=14)
plt.ylabel(r'I(Q)', fontsize=14)
plt.xlim(q[0], q[-1])
#plt.ylim(0, 1.2)
plt.legend(frameon=False)
#plt.title('Wat/Gly 1:1 vol')

plt.tight_layout()
plt.savefig('/Users/maddalena/Desktop/fig1.png', dpi=300)
plt.show()

## Water/Glycerol 1:1 weight

In [ ]:
plt.figure(figsize=(9,4))

plt.subplot(121)
col = plt.cm.jet(np.linspace(1,0,len(Lys_w)))
for i in range(0,len(Lys_w[:-1])):
    #Iq = ang_and_plot(path+'/data/2021-01-14/%s' %WatGlyVol[i])
    plt.plot(q, Lys_w[i], c=col[i], lw=1, label='%d K' %T3[i])

#for i in range(len(Ih_ref)-4):
#    plt.axvline(x=Ih_ref[i], c='gray', lw=1, alpha=1,ls='-.')
plt.xlabel(r'Q [$\rm\AA^{-1}$]', fontsize=14)
plt.ylabel(r'I(Q)', fontsize=14)
plt.xlim(q[0], q[-1])
#plt.ylim(0, 1.2)
plt.legend(frameon=False)
plt.title('lys 200mg/ml in Wat/Gly 1:1 weight')

plt.subplot(122)
col = plt.cm.jet(np.linspace(1,0,len(Solv)+1))
for i in range(0,len(Solv)):
    #Iq = ang_and_plot(path+'/data/2021-01-14/%s' %Lys200Vol[i])
    plt.plot(q, Solv[i], c=col[i], lw=1, label=f'{T2[i]} K')

plt.xlabel(r'Q [$\rm\AA^{-1}$]', fontsize=14)
plt.ylabel(r'I(Q)', fontsize=14)
plt.xlim(q[0], q[-1])
#plt.ylim(0, 1.2)
plt.legend(frameon=False)
#plt.title('Wat/Gly 1:1 vol')

plt.tight_layout()
plt.savefig('/Users/maddalena/Desktop/fig2.png', dpi=300)
plt.show()